## Guillotine Cutting Optimization

In [1]:
using DataStructures

In [2]:
struct Panel
    len::Int
    wid::Int
end

# ist der größere Wert immer die Länge
struct Piece
    len::Int
    wid::Int
    qnt::Int
end

In [3]:
create_id(piece::Piece) = "l$(piece.len)-w$(piece.wid)"

create_id (generic function with 1 method)

In [4]:
pieces = [Piece(3,1,4), Piece(4,2,2), Piece(5,1,1), Piece(2,2,4)]

4-element Vector{Piece}:
 Piece(3, 1, 4)
 Piece(4, 2, 2)
 Piece(5, 1, 1)
 Piece(2, 2, 4)

In [5]:
panel = Panel(5,4)

Panel(5, 4)

In [7]:
p1 = Piece(3,2,1)

Piece(3, 2, 1)

In [8]:
create_id(p1)

"l3-w2"

- start with panels ordered by max len, then max wid, then min len, then min wid, then all random permutations
- keep track of min number of panels & respective list of cuts
- stop exploring solution if exceeds min number of panels OR all pieces cut OR manual interrupt

In [9]:
"calculate the area of a piece"
get_area(piece::Piece) = piece.len * piece.wid

"calculate the area of a panel"
get_area(panel::Panel) = panel.len * panel.wid

get_area

In [10]:
get_area(panel)

20

In [12]:
total_area = sum([get_area(piece)*piece.qnt for piece in pieces])

49

In [13]:
min_panels = ceil(total_area / get_area(panel))

3.0

- take first piece from list
- cut piece from panel: cut panel along one dimension, add 'leftover' to leftover stack, cut second dimension, move piece to "done" stack, move leftover (if exists) to leftover stack
- make sure leftover panel is defined so that longer side is len and shorter is width
- take smallest panel from leftover stack, from first to last, see if any of the remaining pieces in list can be cut from it, if so: cut, move piece to done stack and leftover to leftover stack, if not
move leftover panel to waste
- when leftover panels are empty but there are still pieces: open a new panel

Beginning State:
- list of pieces len > 0
- num panels = 0
- finished pieces empty list
- leftover panels empty list
- list of cuts empty

In [9]:
"Check if a piece can be cut from a panel"
is_cuttable(panel::Panel, piece::Piece) = (piece.len <= panel.len) & (piece.wid <= panel.wid)

is_cuttable

In [12]:
is_cuttable(panel, p1)

true

- how to decide whether to cut len or wid?
- how to record/give out/visualize cuts?
- what if it fits exatly?

In [15]:
"Cut a piece from a panel and return the leftover panels"
function cut(panel::Panel, piece::Piece, dimension::String = "wid") 
    if !is_cuttable(panel, piece)
        throw(DomainError("Piece cannot be cut from panel"))
    end
    if ! (dimension in ["len", "wid"])
        throw(ArgumentError("Dimension must be len or wid"))
    end
    if dimension == "len"
        cut1 = Panel(panel.len - piece.len, panel.wid)
        cut2 = Panel(piece.len, panel.wid - piece.wid)
        return(cut1, cut2)
        end
    if dimension == "wid"
        cut1 = Panel(panel.len, panel.wid - piece.wid)
        cut2 = Panel(panel.len - piece.len, piece.wid)
        return(cut1, cut2)
        end
end

cut

- einfarbige platten (immer so "drehen" dass die längere seite len ist) vs gemaserte Platten (ausrichtung muss fix bleiben)

In [16]:
cut(panel, p1)

(Panel(5, 2), Panel(2, 2))

In [17]:
p2 = Piece(1,1)

Piece(1, 1)

In [18]:
cut(Panel(2,2), Piece(2,2))

(Panel(2, 0), Panel(0, 2))

In [19]:
leftover_exists(leftover::Panel) = leftover.len != 0 && leftover.wid != 0

leftover_exists (generic function with 1 method)

In [22]:
# list of pieces, get total square whatever, how many panels do you need at least?
# preconditions: all pieces can be cut from panel (len <= len, wid <= wid)
piece_list = [Piece(2,2), Piece(3,2), Piece(2,2), Piece(3,2), Piece(3,1)]

5-element Vector{Piece}:
 Piece(2, 2)
 Piece(3, 2)
 Piece(2, 2)
 Piece(3, 2)
 Piece(3, 1)

In [23]:
sort!(piece_list, by= p -> p.wid, rev=true)

5-element Vector{Piece}:
 Piece(2, 2)
 Piece(3, 2)
 Piece(2, 2)
 Piece(3, 2)
 Piece(3, 1)

In [38]:
leftovers = Queue{Panel}()
if is_cuttable(panel, p1)
    lo1, lo2 = cut(panel, p1)
    if leftover_exists(lo1)
        enqueue!(leftovers, lo1)
    end
    if leftover_exists(lo2)
        enqueue!(leftovers, lo2)
    end
end

Queue{Panel}(Deque [Panel[Panel(5, 2), Panel(2, 2)]])

In [44]:
while !isempty(leftovers)
    cur_lo = dequeue!(leftovers)
    for piece in piece_list # filter: qnt > 0
        if is_cuttable(cur_lo, piece)
            # qnt -= 1
        end
    end
end

LoadError: MethodError: no method matching delete!(::Vector{Piece}, ::Piece)
[0mClosest candidates are:
[0m  delete!([91m::Set[39m, ::Any) at set.jl:76
[0m  delete!([91m::IdDict{K}[39m, ::Any) where K at iddict.jl:130
[0m  delete!([91m::OrderedRobinDict[39m, ::Any) at ~/.julia/packages/DataStructures/59MD0/src/ordered_robin_dict.jl:401
[0m  ...